In [7]:
pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------ --------------------------------- 0.7/3.8 MB 20.8 MB/s eta 0:00:01
   -------------------- ------------------- 2.0/3.8 MB 25.5 MB/s eta 0:00:01
   ------------------------------------ --- 3.5/3.8 MB 27.7 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 24.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import lxml
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "http://openapi.seoul.go.kr:8088/666d526447736f6e3436756e6d7844/xml/tbLnOpendataRtmsV/1/5/"
response = requests.get(url)
response

C:\Users\user\AppData\Local\Temp\ipykernel_4752\4062364286.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


<Response [200]>

In [9]:
content = response.content

soup = BeautifulSoup(content, "lxml") # html.parser 
# print(soup)
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <tblnopendatartmsv>
   <list_total_count>
    2589063
   </list_total_count>
   <result>
    <code>
     INFO-000
    </code>
    <message>
     정상 처리되었습니다
    </message>
   </result>
   <row>
    <rcpt_yr>
     2024
    </rcpt_yr>
    <cgg_cd>
     11140
    </cgg_cd>
    <cgg_nm>
     중구
    </cgg_nm>
    <stdg_cd>
     16200
    </stdg_cd>
    <stdg_nm>
     신당동
    </stdg_nm>
    <lotno_se>
     1
    </lotno_se>
    <lotno_se_nm>
     대지
    </lotno_se_nm>
    <mno>
     0432
    </mno>
    <sno>
     1130
    </sno>
    <bldg_nm>
     노블레스빌
    </bldg_nm>
    <ctrt_day>
     20240701
    </ctrt_day>
    <thing_amt>
     25200
    </thing_amt>
    <arch_area>
     59.52
    </arch_area>
    <land_area>
     32.000000
    </land_area>
    <flr>
     4
    </flr>
    <rght_se>
    </rght_se>
    <rtrcn_day>
    </rtrcn_day>
    <arch_yr>
     2012
    </arch_yr>
    <bldg_usg>
     연립다세대
    </bldg_usg>
    <dclr_se>
     직거래
 

C:\Users\user\AppData\Local\Temp\ipykernel_4752\915339795.py:3: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, "lxml") # html.parser


In [10]:
years            = soup.find_all('acc_year')         # 접수년월
sgg_cds          = soup.find_all('sgg_cd')           # 자치구코드
bldg_nms         = soup.find_all('bldg_nm')          # 건물명
obj_amts         = soup.find_all('obj_amt')          # 물건금액(만원)
house_types      = soup.find_all('house_type')       # 건물용도
rdealer_lawdnms  = soup.find_all('rdealer_lawdnm')   # 신고한 개업공인중개사 시군구명

In [11]:
%%time

year_list           = []
sgg_cd_list         = []
bldg_nm_list        = []
obj_amt_list        = []
house_type_list     = []
rdealer_lawdnm_list = []

for year, sgg_cd, bldg_nm, obj_amt, house_type, rdealer_lawdnm in zip(years, sgg_cds, bldg_nms, obj_amts, house_types, rdealer_lawdnms):
  year_list.append(year.get_text())
  sgg_cd_list.append(sgg_cd.get_text())
  bldg_nm_list.append(bldg_nm.get_text())
  obj_amt_list.append(obj_amt.get_text())
  house_type_list.append(house_type.get_text())
  rdealer_lawdnm_list.append(rdealer_lawdnm.get_text())

df = pd.DataFrame({
    "acc_year": year_list, 
    "sgg_cd": sgg_cd_list, 
    "bldg_nm" : bldg_nm_list,
    "obj_amt": obj_amt_list,
    "house_type" : house_type_list,
    "rdealer_lawdnm": rdealer_lawdnm_list
})

df

CPU times: total: 0 ns
Wall time: 0 ns


,acc_year,sgg_cd,bldg_nm,obj_amt,house_type,rdealer_lawdnm


In [12]:
%%time

data = []
for i in range(0,len(years)):
  rows = [years[i].get_text(),
          sgg_cds[i].get_text(), 
          bldg_nms[i].get_text(),
          obj_amts[i].get_text(),
          house_types[i].get_text(),
          rdealer_lawdnms[i].get_text()]
  data.append(rows)

cols = ['acc_year', 'sgg_cd', 'bldg_nm', 'obj_amt', 'house_type', 'rdealer_lawdnm']
df = pd.DataFrame(data, columns = cols)
df

CPU times: total: 0 ns
Wall time: 1.52 ms


,acc_year,sgg_cd,bldg_nm,obj_amt,house_type,rdealer_lawdnm


In [ ]:
#url2 = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?_wadl&type=xml"

In [19]:
url2="http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?_wadl&type=xml&serviceKey=ndDAmw1tVel9ESFs8dZEFH8vPG2k%2B9wh3Hntjz0AWNKTA%2BnrYCR9GEo8kEt%2FK6Zj%2FQnSCnAeNWn9C9aVUW0f6g%3D%3D"
request2 = requests.get(url2)
content2 = request2.content

In [20]:
soup2 = BeautifulSoup(content2, "lxml") # html.parser 
# print(soup)
print(soup2.prettify())

<html>
 <body>
  <application xmlns="http://wadl.dev.java.net/2009/02" xmlns:xs="http://www.w3.org/2001/XMLSchema">
   <grammars>
    <xs:schema attributeformdefault="unqualified" elementformdefault="unqualified" xmlns:xs="http://www.w3.org/2001/XMLSchema">
     <xs:element name="GetRTMSDataSvcAptRentDevVO" type="getRTMSDataSvcAptRentDevVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcAptRentVO" type="getRTMSDataSvcAptRentVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcAptTradeDevVO" type="getRTMSDataSvcAptTradeDevVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcAptTradeVO" type="getRTMSDataSvcAptTradeVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcInduTradeVO" type="getRTMSDataSvcInduTradeVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcLandTradeVO" type="getRTMSDataSvcLandTradeVO">
     </xs:element>
     <xs:element name="GetRTMSDataSvcNrgTradeVO" type="getRTMSDataSvcNrgTradeVO">
     </xs:element>
     <xs:element name="